In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import math

In [3]:
formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"

In [4]:
def getMiscDict(path):
    dirList = path.split('\\')
    syncType = dirList[len(dirList) - 7]
    memoryStorage = dirList[len(dirList) - 6]
    baseline = dirList[len(dirList) -5]
    if baseline.lower() == 'comp':
        baseline = 'TACO'
    elif baseline.lower() == 'async':
        baseline = 'Antifreeze'
    elif baseline.lower() == 'pgimpl':
        baseline = 'NoComp'
    testSpreadsheet = dirList[len(dirList) - 4]
    run = int(dirList[len(dirList) - 3].split('-')[1])
    rows = int(dirList[len(dirList) - 2].split('-')[1])
    return {'Sync/Async':syncType, 'Memory Storage':memoryStorage, 'Baseline':baseline,
            'Test Spreadsheet':testSpreadsheet, 'Run':run, 'Rows':rows}

def getMetrics(path):
    """
    Takes in a path string to a .stat file and returns a Pandas DataFrame of the file's contents
    """
    reportsFile = open(path, 'r')
    dataDict = {}
    dataDict = getMiscDict(path)
    for line in reportsFile:
        if ': ' in line:
            metric, value = line.split(": ")
            if metric not in dataDict:
                dataDict[metric] = []
            try:
                dataDict[metric].append(int(value))
            except ValueError:
                dataDict[metric].append(float(value))
            except:
                dataDict[metric].append(value)
    reportsFile.close
    
    return pd.DataFrame(dataDict)


def getAllMetrics(basePath, miscDict={'sync type':'', 'mem type':'', 'impl':'', 'test':'', 'run':0}):
    """
    Takes in a file path and searches all of its subdirectories for .stat files and returns all of
    their contents in a Pandas DataFrame
    """
    data = pd.DataFrame()
    for file in os.listdir(basePath):
        filePath = os.path.join(basePath, file)
        if os.path.isdir(filePath):
            data = data.append(getAllMetrics(filePath))
        elif file == "core.stat":
            data = data.append(getMetrics(filePath))
    return data


def plotData(data, 
             researchQuestion,
             plotFunc,
             relevantColumns,
             sheets=['Rate', 'RunningTotalFast', 'RunningTotalSlow'], 
             memStorage = ['DB', 'IN_MEM', 'MEM'],
             syncMethod=['async', 'sync'],
             save=False,
             plotHomePath='../plots/Dixin-Exp'):
    plt.rcParams.update({'font.size': 23, 'figure.figsize': (12,8), 'lines.linewidth':3, 'font.family':'serif'})
    for sheet in sheets:
        for sync in syncMethod:
            for mem in memStorage:
                dataSlice = pd.DataFrame()
                try:
                    dataSlice = data.loc[sheet, mem, sync]
                except:
                    print('No data for (' + sheet + ', ' + mem + ', ' + sync + ')')
                
                if len(dataSlice > 0):
                    relevantData = pd.pivot_table(dataSlice, 
                                              index='Rows', 
                                              columns=dataSlice.index)[[relevantColumns]][relevantColumns]
                    relevantData = relevantData[relevantData.columns.tolist()[-2:] + relevantData.columns.tolist()[:-2]]
                    splitData = [relevantData[relevantData.index < 100000], relevantData[relevantData.index >= 100000]]
                    if len(splitData[0]) > 0:
                        plotFunc(splitData[0], researchQuestion, sheet, mem, sync, False, save)
                    if len(splitData[1]) > 0:
                        plotFunc(splitData[1], researchQuestion, sheet, mem, sync, True, save)
                        
def findYMax(data):
    digits = 0
    maxTime = 0
    for column in data.columns:
        maxTime = max(maxTime, data[column].max())
    digitTracker = maxTime
    while digitTracker > 10:
        digits += 1
        digitTracker = digitTracker // 10
    return math.ceil(maxTime / 10 ** digits) * 10**digits + 10 ** (digits - 1)
        
def checkForAntifreeze(df):
    return len(df['Antifreeze']) - df['Antifreeze'].isnull().sum()


def savePlot(plot, homePath, rq, sheet, mem, big):
    filePath = homePath+rq+'/'+sheet+'/'+mem+'/'
    Path(filePath).mkdir(parents=True, exist_ok=True)
    if big:
        filePath += sheet+'_'+mem+'_big.pdf'
    else:
        filePath += sheet+'_'+mem+'_small.pdf'
    plt.savefig(filePath, bbox_inches='tight')
    print('Plot saved to ' + filePath)

In [5]:
#RQ1
def plotTotalTime(plotData, researchQuestion, sheet, mem, sync, big, save):
    plt.figure()
    display(plotData)
    numAntifreeze = checkForAntifreeze(plotData)
    if numAntifreeze == 0:
        plotData.drop(columns='Antifreeze', inplace=True)
    elif numAntifreeze < 5:
        'label last point'
    
    lineplt = sns.lineplot(markers=True, markersize=20, ci=None, dashes=False, alpha=.5, data=plotData)
    lineplt.yaxis.grid()
    lineplt.legend(markerscale=3)
    for index, label in enumerate(lineplt.get_xticklabels()):
        if index % 2 == 0:
            label.set_visible(False)
        else:
            label.set_visible(True)
    lineplt.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_xticks()/1000])
    lineplt.set_ybound(upper=findYMax(plotData))
    if not big and sheet=='RunningTotalSlow':
        lineplt.set_ybound(upper=1000)
    if 0 < numAntifreeze < 5:
        row = plotData.iloc[numAntifreeze - 1]
        lineplt.text(x=int(row.name) + lineplt.get_xlim()[1] * .015, 
                       y=plotData.loc[row.name, 'Antifreeze'] + ((lineplt.get_ylim()[1] - lineplt.get_ylim()[0]) * .03),
                       fontsize=20,
                       s='Did not\nfinish after')
    handles, labels = lineplt.get_legend_handles_labels()
    leg = lineplt.legend(
        bbox_to_anchor=(0, 0, 1, 1),
        loc="upper left",
        handles=handles[0:], 
        labels=labels[0:],
        markerscale=3
        )
    if big: 
        lineplt.set_yticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_yticks()/1000])
    lineplt.set_ylabel('Time of returning control (ms)')
    lineplt.set_xlabel('Number of Rows ')
    if save: 
        savePlot(lineplt, '../plots/Dixin-Exp/', researchQuestion, sheet, mem, big)
        plt.close()
        
        
def plotBars(testData, numRows, big, save=False):
    plt.rcParams.update({'font.size': 40, 'figure.figsize': (12,8), 'lines.linewidth':3, 'font.family':'serif'})
    relevantData = testData[testData['Rows'] == numRows]
    try:
        relevantData = relevantData.loc[:, 'MEM', 'async']
    except:
        relevantData = relevantData.loc[:, 'IN_MEM', 'async']
    relevantData = relevantData[['Total time to update cells (ms)']]
    relevantData = relevantData.reset_index(level=['Test Spreadsheet', 'Baseline']).pivot(index='Test Spreadsheet', columns='Baseline', values='Total time to update cells (ms)')
    relevantData = relevantData[relevantData.columns.tolist()[-2:] + relevantData.columns.tolist()[:-2]]
    display(relevantData)
    ax = relevantData.plot.bar()
    yMax = 200
    ax.set_ylabel('Time of returning control (ms)')
    ax.set_xlabel('')
    ax.set_xticklabels(['Rate', 'Fast', 'Slow'])
    ax.set_ybound(upper=yMax)
    ax.yaxis.grid()
    for label in ax.get_xticklabels():
        label.set_rotation(0)
    for bar in ax.patches:
        if bar.get_height() > yMax:
            ax.annotate('{:.1f}'.format(bar.get_height() / 1000) + '\nsec',
                        (bar.get_x() + bar.get_width() * 1.7, yMax * .9), ha='center', va='center',
                        size=23, xytext=(0, -8),
                        textcoords='offset points',
                        fontsize=40)
    handles, labels = ax.get_legend_handles_labels()
    leg = ax.legend(
        bbox_to_anchor=(.025, .1, 1, 1),
        loc="lower right",
        handles=handles[0:], 
        labels=labels[0:],
        markerscale=3
        )
    
    if save:
        filePath = '../plots/Dixin-Exp/rq1/MemBarPlots/'
        Path(filePath).mkdir(parents=True, exist_ok=True)
        if big:
            plt.savefig(filePath + 'mem_' + 'big.pdf', bbox_inches="tight")
        else:
            plt.savefig(filePath + 'mem_' + 'small.pdf', bbox_inches="tight")
        plt.close()
        
        
formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"
testPath = formulaCompressionTestHome + "\\experiments_data\\rq1\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).mean()
testData = testData.reset_index(level='Rows')
#display(testData)
plotData(testData, 'rq1', plotTotalTime, 'Total time of getting dependents (ms)', memStorage=['DB'], save=True)

plotBars(testData, 10000, False, save=True)
plotBars(testData, 500000, True, save=True)

Baseline,NoComp,TACO,Antifreeze
Rows,,,
5000,67.666667,1.000000,1.0
10000,125.333333,1.666667,1.0
15000,192.000000,1.666667,1.0
20000,253.666667,1.333333,1.0
25000,326.333333,1.333333,NaN


Plot saved to ../plots/Dixin-Exp/rq1/Rate/DB/Rate_DB_small.pdf


<ipython-input-5-f0a1f5bc01e3>:19: UserWarning: FixedFormatter should only be used together with FixedLocator
  lineplt.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_xticks()/1000])


Baseline,NoComp,TACO,Antifreeze
Rows,,,
100000,979.0,1.000000,NaN
200000,2058.0,1.000000,NaN
300000,3160.0,1.333333,NaN
400000,4298.0,1.000000,NaN
500000,5300.0,1.333333,NaN


C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

Baseline,NoComp,TACO,Antifreeze
Rows,,,
100000,3392.333333,1.333333,NaN
200000,5882.666667,1.000000,NaN
300000,8969.666667,1.000000,NaN
400000,13466.333333,1.000000,NaN
500000,18014.333333,1.000000,NaN


C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-5-f0a1f5bc01e3>:19: UserWarning: FixedFormatter should only be used together with FixedLocator
  lineplt.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_xticks()/1000])
<ipython-input-5-f0a1f5bc01e3>:38: UserWarning: FixedFormatter should only be used together with FixedLocator
  lineplt.set_yticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_yticks()/1000])


Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalSlow/DB/RunningTotalSlow_DB_big.pdf
No data for (RunningTotalSlow, DB, sync)


Baseline,NoComp,TACO,Antifreeze
Test Spreadsheet,,,
Rate,162.000000,2.000000,1.000000
RunningTotalFast,142.000000,2.666667,1.666667
RunningTotalSlow,155.666667,2.000000,1.333333


Baseline,NoComp,TACO
Test Spreadsheet,,
Rate,10292.666667,2.666667
RunningTotalFast,6606.333333,2.333333
RunningTotalSlow,9723.333333,3.666667


In [6]:
#RQ2
def plotBatchLoadingTime(plotData, researchQuestion, sheet, mem, sync, big, save):
    display(plotData)
    plt.figure()
    yMax = findYMax(plotData[['NoComp', 'TACO']])
    if sheet == 'RunningTotalFast':
        yMax = 80000
    numAntifreeze = checkForAntifreeze(plotData)
    if numAntifreeze == 0:
        plotData.drop(columns='Antifreeze', inplace=True)
    cappedData = plotData.copy()
    for column in plotData.columns:
        cappedData[column] = plotData[column].map(lambda x: min(x, yMax * .95))
    batchPlot = sns.lineplot(markers=True, markersize=20, ci=None, dashes=False, alpha=.5, color="black", data=cappedData)

    for index, label in enumerate(batchPlot.get_xticklabels()):
        if index % 2 == 0:
            label.set_visible(False)
        else:
            label.set_visible(True)
    batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
    if 0 < numAntifreeze < 5:
        row = plotData.iloc[numAntifreeze - 1]
        batchPlot.text(x=int(row.name) + batchPlot.get_xlim()[1] * .015, 
                       y=cappedData.loc[row.name, column] - (batchPlot.get_ylim()[1] + batchPlot.get_ylim()[0]) * .045,
                       fontsize=20,
                       s='Did not\nfinish after')
    batchPlot.yaxis.grid()
    batchPlot.set_ybound(upper=yMax)
    xAdjust = - max(plotData.index.values) * .03
    yAdjust = yMax * .1
    xMax = max(plotData.index.values)
    for column in plotData.columns:
        for i, row in plotData[plotData[column] > cappedData[column]].iterrows():
            batchPlot.text(x=int(row.name) + xAdjust, 
                           y=cappedData.loc[row.name, column] - yAdjust, 
                           s='{:,.1f}'.format(row[column] / 60000) + ' min')
    batchPlot.legend(markerscale=3)
    handles, labels = batchPlot.get_legend_handles_labels()
    leg = batchPlot.legend(
        bbox_to_anchor=(.57, .1, 1, 1),
        loc="lower left",
        handles=handles[0:], 
        labels=labels[0:],
        markerscale=3
        )
    batchPlot.set_yticklabels(['{:.0f}'.format(x) + 'K' for x in batchPlot.get_yticks()/1000])
    batchPlot.set_ylabel('Time of building the graph (ms)')
    batchPlot.set_xlabel('Number of Rows')
    if save: 
        savePlot(batchPlot, '../plots/Dixin-Exp/', researchQuestion, sheet, mem, big)
        plt.close()
        
        
        
        
        
formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"
testPath = formulaCompressionTestHome + "\\experiments_data\\rq2\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).mean()
testData = testData.reset_index(level='Rows')
#display(testData)

plotData(testData, 'rq2', plotBatchLoadingTime, 'Total time of adding the batch (ms)', memStorage=['DB'], save=True)



Baseline,NoComp,TACO,Antifreeze
Rows,,,
5000,8065.333333,37.333333,26888.333333
10000,19865.666667,63.000000,94770.333333
15000,30636.666667,65.333333,458252.333333
20000,44446.333333,80.666667,867453.000000
25000,56966.666667,89.333333,NaN


Plot saved to ../plots/Dixin-Exp/rq2/Rate/DB/Rate_DB_small.pdf


<ipython-input-6-77a793842164>:21: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
<ipython-input-6-77a793842164>:47: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_yticklabels(['{:.0f}'.format(x) + 'K' for x in batchPlot.get_yticks()/1000])


Baseline,NoComp,TACO,Antifreeze
Rows,,,
100000,159493.0,192.666667,NaN
200000,299052.0,292.000000,NaN
300000,490979.0,393.666667,NaN
400000,596563.0,483.000000,NaN
500000,748533.0,499.666667,NaN


C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-6-77a793842164>:21: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
<ipython-input-6-77a793842164>:47: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_yticklabels(['{:.0f}'.format(x) + 'K' for x in batchPlot.get_yticks()/1000])


...

Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalSlow/DB/RunningTotalSlow_DB_small.pdf


<ipython-input-6-77a793842164>:47: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_yticklabels(['{:.0f}'.format(x) + 'K' for x in batchPlot.get_yticks()/1000])


Baseline,NoComp,TACO,Antifreeze
Rows,,,
100000,83338.000000,94.333333,NaN
200000,166016.000000,126.666667,NaN
300000,229874.000000,175.666667,NaN
400000,310085.666667,164.666667,NaN
500000,386317.666667,228.333333,NaN


C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-6-77a793842164>:21: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
<ipython-input-6-77a793842164>:47: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_yticklabels(['{:.0f}'.format(x) + 'K' for x in batchPlot.get_yticks()/1000])


Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalSlow/DB/RunningTotalSlow_DB_big.pdf
No data for (RunningTotalSlow, DB, sync)


In [47]:
#RQ3
def plotRQ3(plotData, researchQuestion, sheet, mem, sync, big, save):
    display(plotData)
    plt.figure()
    if big:
        plotData.iloc[1, 0] = float('nan')
        plotData.iloc[2, 0] = float('nan')
    numAntifreeze = checkForAntifreeze(plotData)
    if numAntifreeze == 0:
        plotData.drop(columns='Antifreeze', inplace=True)
    elif numAntifreeze < 5:
        'label last point'
    cappedData = plotData.copy()
    for column in plotData.columns:
        plotData[column] = plotData[column] / 1000
    yMax = 100
    for column in plotData.columns:
        cappedData[column] = plotData[column].map(lambda x: min(x, yMax * .95))
    batchPlot = sns.lineplot(markers=True, markersize=35, ci=None, dashes=False, alpha=.5, color="black", data=cappedData)
    #batchPlot.set_xticklabels(['{:.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
    if big:
        batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in range(0, 501, 100)])
    else:
        batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in range(0, 11, 2)])
    batchPlot.yaxis.grid()
    xAdjust = - max(plotData.index.values) * .03
    yAdjust = yMax * .1
    xMax = max(plotData.index.values)
    for column in plotData.columns:
        for i, row in plotData[plotData[column] > cappedData[column]].iterrows():
            xMax = max(plotData.index.values) * 1.14
            batchPlot.text(x=int(row.name) + xAdjust, 
                           y=cappedData.loc[row.name, column] - yAdjust, 
                           s='{:.1f}'.format(row[column] / 60) + ' min',
                          fontsize=25)
    batchPlot.set_yticklabels(['{:.0f}'.format(x) + 'K' for x in batchPlot.get_yticks()])
    batchPlot.set_ybound(upper=yMax)
    batchPlot.set_xbound(upper=xMax)
    batchPlot.legend(markerscale=5)
    batchPlot.set_ylabel('Time of finishing the modification (ms)')
    batchPlot.set_xlabel('Number of Rows Modified')
    if big:
        display(cappedData)
        batchPlot.text(x=100000 + batchPlot.get_xlim()[1] * .03, 
                       y=95.0 - (batchPlot.get_ylim()[1] + batchPlot.get_ylim()[0]) * .045,
                       fontsize=20,
                       s='Did not\nfinish after')
    if save: 
        savePlot(batchPlot, '../plots/Dixin-Exp/', researchQuestion, sheet, mem, big)
        plt.close()


testPath = formulaCompressionTestHome + "\\experiments_data\\rq3\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).mean()
testData = testData.reset_index(level='Rows')

plotData(testData, 'rq3', plotRQ3, 'Total time of refreshing the cache (ms)', sheets=['RefreshCache'], memStorage=['DB', 'MEM'], save=True)

Baseline,NoComp,TACO,Antifreeze
Rows,,,
2000,7744.0,278.666667,744962.0
4000,11460.0,474.666667,795208.0
6000,27402.0,751.000000,844966.0
8000,35325.0,833.666667,877087.0
10000,44308.0,1019.666667,928016.0


Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/DB/RefreshCache_DB_small.pdf


<ipython-input-47-a9ce240d1cf2>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-47-a9ce240d1cf2>:24: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in range(0, 11, 2)])
<ipython-input-47-a9ce240d1cf2>:36: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_yticklabels(['{:.0f}'.format(x) + 'K' for x in batchPlot.get_yticks()])


Baseline,NoComp,TACO,Antifreeze
Rows,,,
100000,1698297.0,7537.666667,NaN
200000,3818311.0,16265.666667,NaN
300000,6172711.0,21775.333333,NaN
400000,NaN,28923.666667,NaN
500000,NaN,40823.333333,NaN


C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

...

Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/MEM/RefreshCache_MEM_small.pdf


Baseline,NoComp,TACO,Antifreeze
Rows,,,
100000,NaN,9536.666667,NaN
200000,NaN,14229.000000,NaN
300000,NaN,22676.000000,NaN
400000,NaN,30132.333333,NaN
500000,NaN,69960.666667,NaN


C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

Baseline,NoComp,TACO
Rows,,
100000,NaN,9.536667
200000,NaN,14.229000
300000,NaN,22.676000
400000,NaN,30.132333
500000,NaN,69.960667


Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/MEM/RefreshCache_MEM_big.pdf
No data for (RefreshCache, DB, sync)
No data for (RefreshCache, MEM, sync)


In [21]:

testPaths = [formulaCompressionTestHome + "\\experiments_data\\rq1\\report",
             formulaCompressionTestHome + "\\experiments_data\\rq2\\report",
             formulaCompressionTestHome + "\\experiments_data\\rq3\\report"]
for testPath in testPaths:
    testData = getAllMetrics(testPath)
    testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).mean()
    testData = testData.groupby(['Memory Storage', 'Baseline', 'Rows']).max()
    display(testData)


Run  Total test time (ms)  \
Memory Storage Baseline   Rows                                
DB             Antifreeze 5000    2.0          4.465110e+05   
                          10000   2.0          1.113132e+06   
                          15000   2.0          3.094977e+06   
                          20000   2.0          2.526406e+06   
               NoComp     5000    2.0          1.667400e+04   
                          10000   2.0          3.531833e+04   
                          15000   2.0          6.615833e+04   
                          20000   2.0          8.350500e+04   
                          25000   2.0          1.126023e+05   
                          100000  2.0          3.153820e+05   
                          200000  2.0          5.926400e+05   
                          300000  2.0          8.569450e+05   
                          400000  2.0          1.182612e+06   
                          500000  2.0          1.443307e+06   
               TACO       5000    2.0          6.797333e+03   
                          10000   2.0          1.580500e+04   
                          15000   2.0          2.334133e+04   
                          20000   2.0          3.235333e+04   
                          25000   2.0          4.346533e+04   
                          100000  2.0          1.226040e+05   
                          200000  2.0          2.410597e+05   
                          300000  2.0          3.639830e+05   
                          400000  2.0          4.952907e+05   
                          500000  2.0          6.221847e+05   
MEM            Antifreeze 10000   2.0          1.216398e+06   
               NoComp     10000   2.0          3.482033e+04   
                          500000  2.0          1.685304e+06   
               TACO       10000   2.0          1.434100e+04   
                          500000  2.0          7.284480e+05   

                                  Number of cells to update  \
Memory Storage Baseline   Rows                                
DB             Antifreeze 5000                       5001.0   
                          10000                     10001.0   
                          15000                     15001.0   
                          20000                     20001.0   
               NoComp     5000                       5001.0   
                          10000                     10001.0   
                          15000                     15001.0   
                          20000                     20001.0   
                          25000                     25001.0   
                          100000                   100001.0   
                          200000                   200001.0   
                          300000                   300001.0   
                          400000                   400001.0   
                          500000                   500001.0   
               TACO       5000                       5001.0   
                          10000                     10001.0   
                          15000                     15001.0   
                          20000                     20001.0   
                          25000                     25001.0   
                          100000                   100001.0   
                          200000                   200001.0   
                          300000                   300001.0   
                          400000                   400001.0   
                          500000                   500001.0   
MEM            Antifreeze 10000                     10001.0   
               NoComp     10000                     10001.0   
                          500000                   500001.0   
               TACO       10000                     10001.0   
                          500000                   500001.0   

                                  Number of cells updated  \
Memory Storage Baseline   Rows                              
DB  

Run  Total test time (ms)  \
Memory Storage Baseline   Rows                                
DB             Antifreeze 5000    2.0          4.465110e+05   
                          10000   2.0          1.113132e+06   
                          15000   2.0          3.094977e+06   
                          20000   2.0          2.526406e+06   
               NoComp     5000    2.0          1.667400e+04   
                          10000   2.0          3.531833e+04   
                          15000   2.0          6.615833e+04   
                          20000   2.0          8.350500e+04   
                          25000   2.0          1.126023e+05   
                          100000  2.0          3.153820e+05   
                          200000  2.0          5.926400e+05   
                          300000  2.0          8.569450e+05   
                          400000  2.0          1.182612e+06   
                          500000  2.0          1.443307e+06   
               TACO       5000    2.0          6.797333e+03   
                          10000   2.0          1.580500e+04   
                          15000   2.0          2.334133e+04   
                          20000   2.0          3.235333e+04   
                          25000   2.0          4.346533e+04   
                          100000  2.0          1.226040e+05   
                          200000  2.0          2.410597e+05   
                          300000  2.0          3.639830e+05   
                          400000  2.0          4.952907e+05   
                          500000  2.0          6.221847e+05   

                                  Number of cells to update  \
Memory Storage Baseline   Rows                                
DB             Antifreeze 5000                       5001.0   
                          10000                     10001.0   
                          15000                     15001.0   
                          20000                     20001.0   
               NoComp     5000                       5001.0   
                          10000                     10001.0   
                          15000                     15001.0   
                          20000                     20001.0   
                          25000                     25001.0   
                          100000                   100001.0   
                          200000                   200001.0   
                          300000                   300001.0   
                          400000                   400001.0   
                          500000                   500001.0   
               TACO       5000                       5001.0   
                          10000                     10001.0   
                          15000                     15001.0   
                          20000                     20001.0   
                          25000                     25001.0   
                          100000                   100001.0   
                          200000                   200001.0   
                          300000                   300001.0   
                          400000                   400001.0   
                          500000                   500001.0   

                                  Number of cells updated  \
Memory Storage Baseline   Rows                              
DB             Antifreeze 5000                        0.0   
                          10000                       0.0   
                          15000                       0.0   
                          20000                       0.0   
               NoComp     5000                        0.0   
                          10000                       0.0   
                          15000                       0.0   
                          20000                       0.0   
                          25000                       0.0   
                          100000                      0.0   
                        

Run  Total test time (ms)  \
Memory Storage Baseline   Rows                                
DB             Antifreeze 2000    2.0          1.186802e+06   
                          4000    2.0          1.338883e+06   
                          6000    2.0          1.352203e+06   
                          8000    2.0          1.232627e+06   
                          10000   2.0          1.338157e+06   
               NoComp     2000    2.0          2.993100e+04   
                          4000    2.0          3.562500e+04   
                          6000    2.0          6.278700e+04   
                          8000    2.0          6.118600e+04   
                          10000   2.0          8.800100e+04   
                          100000  2.0          3.193759e+06   
                          200000  1.0          6.113024e+06   
                          300000  1.0          1.018877e+07   
               TACO       2000    2.0          1.121833e+04   
                          4000    2.0          1.616467e+04   
                          6000    2.0          2.272067e+04   
                          8000    2.0          2.505333e+04   
                          10000   2.0          3.387833e+04   
                          100000  2.0          6.004307e+05   
                          200000  2.0          8.548867e+05   
                          300000  2.0          1.157702e+06   
                          400000  2.0          1.377761e+06   
                          500000  2.0          1.766491e+06   
MEM            Antifreeze 2000    2.0          1.084439e+06   
                          4000    2.0          1.296532e+06   
                          6000    2.0          1.317007e+06   
                          8000    2.0          1.291936e+06   
                          10000   2.0          1.347207e+06   
               NoComp     2000    2.0          5.514333e+04   
                          4000    2.0          6.885300e+04   
                          6000    2.0          9.017833e+04   
                          8000    2.0          9.173300e+04   
                          10000   2.0          9.200133e+04   
               TACO       2000    2.0          1.168200e+04   
                          4000    2.0          1.538833e+04   
                          6000    2.0          2.238767e+04   
                          8000    2.0          2.647133e+04   
                          10000   2.0          4.068033e+04   
                          100000  2.0          6.017250e+05   
                          200000  2.0          8.100557e+05   
                          300000  2.0          9.640687e+05   
                          400000  2.0          1.200726e+06   
                          500000  2.0          1.734424e+06   

                                  Number of cells to update  \
Memory Storage Baseline   Rows                                
DB             Antifreeze 2000                      10001.0   
                          4000                      10001.0   
                          6000                      10001.0   
                          8000                      10001.0   
                          10000                     10001.0   
               NoComp     2000                      10001.0   
                          4000                      10001.0   
                          6000                      10001.0   
                          8000                      10001.0   
                          10000                     10001.0   
                          100000                   500001.0   
                          200000                   500001.0   
                          300000                   500001.0   
               TACO       2000                      10001.0   
                          4000                      10001.0   
                          6000                      10001.0   
                          8000                      10001.0   
 